In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
print(tf.__version__)

2.10.1


In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [48]:
lst = []

with open('../data/full_tab_string.txt') as small_pf:

    tmp_list = []
    for line in small_pf:
        line = line.rstrip("\n")
        if line == "":
            lst.append(tmp_list)
            tmp_list = []
        else:
            tmp_list.extend(line.split())

    if tmp_list:  # add last one
        lst.append(tmp_list)
lst = lst[0]

In [49]:
text = ""
with open('../data/full_tab_string.txt') as small_pf:
    text = small_pf.read()

In [50]:
#Can't use set because tab encodings are not hashable types
def unique(list1):
    unique_set = set()
    for x in list1:
        if x not in unique_set:
            unique_set.add(x)
    return unique_set

In [51]:
#list of all tab locations
tab_vocab = unique(lst)

In [52]:
#this is a quick fix and needs to be done at the raw data level
new_list = []
for tab in tab_vocab:
    if int(tab[1:]) <= 22:
        new_list.append(tab)
tab_vocab = new_list

In [53]:
ids_from_chars = tf.keras.layers.StringLookup(
    num_oov_indices = 1, 
    vocabulary=list(tab_vocab), 
    mask_token=None)
print(ids_from_chars.get_vocabulary())

['[UNK]', 'E10', 'B0', 'A6', 'G17', 'E17', 'D15', 'e10', 'G12', 'D11', 'E11', 'A12', 'G4', 'D6', 'G19', 'e21', 'A0', 'B5', 'A5', 'A1', 'G9', 'G21', 'A3', 'e11', 'e12', 'A15', 'D13', 'E1', 'e19', 'B8', 'B12', 'G18', 'e16', 'D17', 'e13', 'E19', 'D8', 'A4', 'e5', 'B9', 'G2', 'A7', 'E8', 'E5', 'E14', 'G15', 'A14', 'e0', 'e9', 'D1', 'e22', 'e2', 'B14', 'B17', 'B11', 'D9', 'D4', 'B7', 'B3', 'G3', 'E6', 'e1', 'B2', 'D12', 'e7', 'G7', 'G5', 'A11', 'B4', 'e18', 'e6', 'E3', 'D16', 'B19', 'D10', 'e20', 'D2', 'e3', 'G16', 'D7', 'E12', 'e15', 'B10', 'E9', 'e14', 'G0', 'A13', 'B1', 'D0', 'G6', 'G8', 'E2', 'E7', 'A2', 'A8', 'e8', 'e4', 'e17', 'D19', 'B21', 'D3', 'E0', 'B13', 'G1', 'G14', 'B20', 'A9', 'G10', 'A10', 'B18', 'D14', 'G11', 'D5', 'E4', 'B15', 'B16']


In [54]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [55]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids) + " ", axis=-1)

In [56]:
all_ids = ids_from_chars(tf.strings.split(text))

In [57]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
type(ids_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [58]:
seq_length = 10

In [59]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
print(type(sequences))
for seq in sequences.take(1):
  print(chars_from_ids(seq))
  print(type(seq))

<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
tf.Tensor([b'D7' b'G5' b'B5' b'e5' b'e7' b'D6' b'B5' b'G5' b'e7' b'e8' b'D5'], shape=(11,), dtype=string)
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [60]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [61]:
dataset = sequences.map(split_input_target)
dataset



<MapDataset element_spec=(TensorSpec(shape=(10,), dtype=tf.int64, name=None), TensorSpec(shape=(10,), dtype=tf.int64, name=None))>

In [62]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'D7 G5 B5 e5 e7 D6 B5 G5 e7 e8 '
Target: b'G5 B5 e5 e7 D6 B5 G5 e7 e8 D5 '


In [63]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 10), dtype=tf.int64, name=None), TensorSpec(shape=(64, 10), dtype=tf.int64, name=None))>

In [64]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 1024

In [65]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)

    #adjust the dense units size                         
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [66]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [67]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 10, 116) # (batch_size, sequence_length, vocab_size)


In [68]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  7424      
                                                                 
 gru (GRU)                   multiple                  3348480   
                                                                 
 dense (Dense)               multiple                  118900    
                                                                 
Total params: 3,474,804
Trainable params: 3,474,804
Non-trainable params: 0
_________________________________________________________________


In [69]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [70]:
sampled_indices

array([ 87,   2,  92,  73,  92, 102,  32,  71, 102,  88], dtype=int64)

In [71]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [72]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 10, 116)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.7535563, shape=(), dtype=float32)


In [73]:
model.compile(optimizer='adam', loss=loss)

In [74]:
# Directory where the checkpoints will be saved
checkpoint_dir = './tab_training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [75]:
EPOCHS = 10

In [82]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
411/411 [==============================] - 3s 6ms/step - loss: 0.7778
Epoch 2/10
411/411 [==============================] - 2s 6ms/step - loss: 0.7715
Epoch 3/10
411/411 [==============================] - 2s 6ms/step - loss: 0.7648
Epoch 4/10
411/411 [==============================] - 3s 6ms/step - loss: 0.7598
Epoch 5/10
411/411 [==============================] - 3s 6ms/step - loss: 0.7549
Epoch 6/10
411/411 [==============================] - 2s 6ms/step - loss: 0.7517
Epoch 7/10
411/411 [==============================] - 2s 6ms/step - loss: 0.7470
Epoch 8/10
411/411 [==============================] - 3s 6ms/step - loss: 0.7443
Epoch 9/10
411/411 [==============================] - 2s 6ms/step - loss: 0.7419
Epoch 10/10
411/411 [==============================] - 3s 6ms/step - loss: 0.7378


In [83]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, omit, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.

    skip_ids = self.ids_from_chars(omit)[:, None]

    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, omit, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]

    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.

    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [84]:
from MidiToTabOptions import *
tab_midi_dict = generate_tab_midi_pairs()

In [85]:
#Array of Midi notes
notes = [60, 61, 63, 60, 56, 58]
states = None

#First tab note, hard coded
base_tab = 'D8 '

#Dynamically starting input 
append_tab = tf.constant([base_tab])
for note in notes:

    #Finds all tab options for the current note and 
    #removes them from the omitted option list in the OneStep function
    options = ['[UNK]']
    options.extend(tab_midi_dict[note])
    omit = list(set(ids_from_chars.get_vocabulary()) - set(options))
    omit.insert(0, '[UNK]')

    omit_sorted = []
    for item in ids_from_chars.get_vocabulary():
        if item in omit:
            omit_sorted.append(item)

    #generates the next predicted tab note out of the set of possible options
    tf.keras.backend.clear_session()
    one_step_model = OneStep(model, chars_from_ids, ids_from_chars, omit_sorted)

    #bootstrap the predictions 
    bootstrap_list = []    
    for i in range(100):
        next_char, states = one_step_model.generate_one_step(append_tab, omit_sorted, states=states)    
        bootstrap_list.append(next_char.numpy()[0].decode('utf-8'))
    next_tab = max(set(bootstrap_list), key=bootstrap_list.count)


    #append new tab to input list
    append_tab = tf.constant([append_tab.numpy()[0].decode('utf-8') + next_tab + ' '])


In [86]:
string = str(append_tab.numpy()[0].decode('utf-8'))
array = string.split(' ')[:-1]
print(convert_array_to_tab(array))

e|---------------------
B|---------4-----------
G|---5--6-----5-----3--
D|8--------------------
A|---------------11----
E|---------------------



In [46]:
print(append_tab)

tf.Tensor([b'D8 G5 B2 B4 G5 A11 D8 '], shape=(1,), dtype=string)
